# Simple Linear Regression

In this notebook we will use data on house sales in King County to predict house prices using simple (one input) linear regression. You will:
* Use the pandas library to load existing dataset
* Write a function to compute the Simple Linear Regression weights using the closed form solution
* Write a function to make predictions of the output given the input feature
* Turn the regression around to predict the input given the output
* Compare two different models for predicting house prices

In this notebook you will be provided with some already complete code as well as some code that you should complete yourself in order to answer quiz questions. The code we provide to complte is optional and is there to assist you with solving the problems but feel free to ignore the helper code and write your own.

## Import the libraries

In [ ]:
import sklearn, pandas
import numpy as np

## Load house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [ ]:
full_data = pandas.read_csv(r"/content/drive/MyDrive/FUNIX Progress/MLP302x_1.1-A_EN/data/kc_house_data.csv", index_col=0)
full_data.head()

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
id,,,,,,,,,,,,,,,,,,,,
7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


## View the read data

We should split the data into train / test set. To maintain runtime consistency, we will need a seed value (default = 0)

In [ ]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(full_data, train_size=0.8, test_size=0.2, random_state=0)

## Useful Pandas functions

In order to make use of the closed form solution as well as taking advantage of pandas's built-in capacity, we will do some simple exercise. In particular:
* Computing the sum of a column
* Computing the arithmetic average (mean) of the columns
* multiplying and updating a column with a constant
* multiplying a column with another column's value

In [ ]:
# Let's compute the mean of the House Prices in King County in 2 different ways.
prices = full_data['price']

# recall that the arithmetic average (the mean) is the sum of the prices divided by the total number of houses:
avg_price_auto = prices.mean()
avg_price_manual = sum(prices.values) / float(len(prices))
print("average price (function): %.2f" % (avg_price_auto))
print("average price (manual):  %.2f" % (avg_price_manual))

average price (function): 540088.14
average price (manual):  540088.14


In [ ]:
# if we want to multiply every price by 0.5 it's a simple as:
half_prices = 0.5*prices
# Let's compute the sum of squares of price. We can multiply two SArrays of the same length elementwise also with *
prices_squared = prices*prices
sum_prices_squared = prices_squared.sum() # price_squared is an SArray of the squares and we want to add them up.
print("the sum of price squared is: %.2E" % (sum_prices_squared))

the sum of price squared is: 9.22E+15


In [ ]:
print("the sum of price squared is: %.4f" % (sum_prices_squared))

the sum of price squared is: 9217325138472070.0000


Aside: The python notation x.xxe+yy means x.xx \* 10^(yy). e.g 100 = 10^2 = 1*10^2 = 1e2

Replace the exponential notation (%.2E) with the float notation (%.4f) and see the display difference

## Build a generic simple linear regression function 

Armed with these functions we can use the closed form solution found from lecture to compute the slope and intercept for a simple linear regression on observations stored as DataFrame columns: input_feature, output.

Complete the following function (or write your own) to compute the simple linear regression slope and intercept:

In [ ]:
# Mean Method

def simple_linear_regression(input_feature, output):
  mean_XY = (input_feature * output).mean()
  mean_X = input_feature.mean()
  mean_Y = output.mean()
  
  square_X = input_feature * input_feature
  mean_squareX = square_X.mean()

  numerator = (mean_XY) - (mean_X)*(mean_Y)
  denominator = (mean_squareX) - (mean_X)*(mean_X)

  slope = numerator / denominator
  intercept = (mean_Y) - slope * (mean_X)

  return (intercept, slope)

We can test that our function works by passing it something where we know the answer. In particular we can generate a feature and then put the output exactly on a line: output = 1 + 1\*input_feature then we know both our slope and intercept should be 1

In [ ]:
mock_feature = np.array(range(5))
mock_output = 1 + 1*mock_feature
(mock_intercept, mock_slope) =  simple_linear_regression(mock_feature, mock_output)
print("Intercept: %.2f" % (mock_intercept))
print("Slope: %.2f" % (mock_slope))

Intercept: 1.00
Slope: 1.00


Now that we know it works let's build a regression model for predicting price based on sqft_living. Rembember that we train on train_data!

In [ ]:
sqft_features = train_data['sqft_living'].values
sqft_labels = train_data['price'].values
sqft_intercept, sqft_slope = simple_linear_regression(sqft_features, sqft_labels)

print("Intercept: %.2f" % (sqft_intercept))
print("Slope: %.2f" % (sqft_slope))

Intercept: -48257.06
Slope: 283.97


### Define my own function


In [ ]:
from scipy import stats

def self_simple_linear_regression(input_feature, output):
  std_X = np.std(input_feature)
  std_Y = np.std(output)

  mean_X = input_feature.mean()
  mean_Y = output.mean()

  pearson = stats.pearsonr(input_feature, output)[0]

  slope = pearson * (std_Y / std_X)
  intercept = mean_Y - slope * mean_X

  return (intercept, slope)

In [ ]:
self_mock_feature = np.array(range(5))
self_mock_output = 1 + 1*self_mock_feature
(self_mock_intercept, self_mock_slope) = self_simple_linear_regression(self_mock_feature, self_mock_output)
print("Intercept: %.2f" % (self_mock_intercept))
print("Slope: %.2f" % (self_mock_slope))

Intercept: 1.00
Slope: 1.00


# Predicting Values

Now that we have the model parameters: intercept & slope we can make predictions. Using numpy, it's easy to multiply a numpy array by a constant and add a constant value. Complete the following function to return the predicted output given the input_feature, slope and intercept:

In [ ]:
def get_regression_predictions(input_feature, intercept, slope):
    # calculate the predicted values:
    predicted_values = intercept + input_feature * slope
    return predicted_values

Now that we can calculate a prediction given the slope and intercept let's make a prediction. Use (or alter) the following to find out the estimated price for a house with 2000 squarefeet according to the squarefeet model we estimated above.

**Quiz Question: Using your Slope and Intercept from (4), What is the predicted price for a house with 2000 sqft?**

In [ ]:
my_house_sqft = 2000
estimated_price = get_regression_predictions(my_house_sqft, sqft_intercept, sqft_slope)
print("The estimated price for a house with %d squarefeet is $%.4f" % (my_house_sqft, estimated_price))

The estimated price for a house with 2000 squarefeet is $519680.0507


In [ ]:
full_data[full_data['sqft_living'] == 2000][['price']]

,price
id,
1525200060,577500.0
4040800810,420000.0
5693500270,715000.0
108000127,456500.0
1624079104,540000.0
...,...
8857600540,265000.0
984100450,295000.0
2143700406,300000.0


# Residual Sum of Squares

Now that we have a model and can make predictions let's evaluate our model using Residual Sum of Squares (RSS). Recall that RSS is the sum of the squares of the residuals and the residuals is just a fancy word for the difference between the predicted output and the true output. 

Complete the following (or write your own) function to compute the RSS of a simple linear regression model given the input_feature, output, intercept and slope:

In [ ]:
def get_residual_sum_of_squares(input_feature, output, intercept, slope):
    # First get the predictions
    # prediction = get_regression_predictions(input_feature, intercept, slope)
    prediction = intercept + input_feature * slope

    # then compute the residuals (since we are squaring it doesn't matter which order you subtract)
    residuals = output - prediction 

    # square the residuals and add them up
    RSS = sum(residuals ** 2)

    return(RSS)

Let's test our `get_residual_sum_of_squares` function by applying it to the test model where the data lie exactly on a line. Since they lie exactly on a line the residual sum of squares should be zero!

In [ ]:
print("%.2f" % get_residual_sum_of_squares(mock_feature, mock_output, mock_intercept, mock_slope)) # should be 0.0

0.00


Now use your function to calculate the RSS on training data from the squarefeet model calculated above.

**Quiz Question: According to this function and the slope and intercept from the squarefeet model, what is the RSS for the simple linear regression using squarefeet to predict prices on TRAINING data?**

In [ ]:
# You have made your bed, time to lie in it
train_features = train_data['sqft_living'].values
train_labels = train_data['price']
rss_prices_on_sqft = get_residual_sum_of_squares(train_features, train_labels, sqft_intercept, sqft_slope)
print('The RSS of predicting Prices based on Square Feet is : %.2E' % (rss_prices_on_sqft))

The RSS of predicting Prices based on Square Feet is : 1.21E+15


In [ ]:
print('The RSS of predicting Prices based on Square Feet is : %.4f' % (rss_prices_on_sqft))

The RSS of predicting Prices based on Square Feet is : 1209822763116213.5000


# Predict the squarefeet given price

What if we want to predict the squarefoot given the price? Since we have an equation $y = a + b*x$ we can solve the function for x. So that if we have the intercept (a) and the slope (b) and the price (y) we can solve for the estimated squarefeet (x).

Complete the following function to compute the inverse regression estimate, i.e. predict the input_feature given the output.

In [ ]:
def inverse_regression_predictions(output, intercept, slope):
    # solve output = intercept + slope*input_feature for input_feature. Use this equation to compute the inverse predictions:
    estimated_feature = (output - intercept) / slope
    return estimated_feature

Now that we have a function to compute the squarefeet given the price from our simple regression model let's see how big we might expect a house that costs $800,000 to be.

**Quiz Question: According to this function and the regression slope and intercept from (3) what is the estimated square-feet for a house costing $800,000?**

In [ ]:
# instant result, just add your parameters in the function below
my_house_price = 800000
estimated_squarefeet = inverse_regression_predictions(my_house_price, sqft_intercept, sqft_slope)
print("The estimated squarefeet for a house worth $%.2f is %d" % (my_house_price, estimated_squarefeet))

The estimated squarefeet for a house worth $800000.00 is 2987


In [ ]:
full_data[full_data['price'] == my_house_price][['sqft_living']].head()

,sqft_living
id,
326069104,3830
7574910860,2570
3456000160,2380
3623500135,2350
9839300875,1700


# New Model: estimate prices from bedrooms

We have made one model for predicting house prices using squarefeet, but there are many other features in the sales data. 
Use your simple linear regression function to estimate the regression parameters from predicting Prices based on number of bedrooms. Use the training data!

In [ ]:
# Estimate the slope and intercept for predicting 'price' based on 'bedrooms'. Ctrl+C and Ctrl+V is your friend
bedrooms_features = train_data['bedrooms'].values
bedrooms_labels = train_data['price'].values
bedrooms_intercept, bedrooms_slope = simple_linear_regression(bedrooms_features, bedrooms_labels)

print("Intercept: %.2f" % (bedrooms_intercept))
print("Slope: %.2f" % (bedrooms_slope))

Intercept: 126751.85
Slope: 123535.46


# Test your Linear Regression Algorithm

Now we have two models for predicting the price of a house. How do we know which one is better? Calculate the RSS on the TEST data (remember this data wasn't involved in learning the model). Compute the RSS from predicting prices using bedrooms and from predicting prices using squarefeet.

**Quiz Question: Which model (square feet or bedrooms) has lowest RSS on TEST data? Think about why this might be the case.**

In [ ]:
# Compute RSS when using bedrooms on TEST data. Handholding level: high.
bdrm_test_features = test_data['bedrooms'].values
bdrm_test_labels = test_data['price'].values
rss_prices_on_bedroom = get_residual_sum_of_squares(bdrm_test_features, bdrm_test_labels, bedrooms_intercept, bedrooms_slope)
print("Bedroom regressions result: %.2E" % rss_prices_on_bedroom)

Bedroom regressions result: 4.73E+14


In [ ]:
print("Bedroom regressions result: %.4f" % rss_prices_on_bedroom)

Bedroom regressions result: 472745600358877.7500


In [ ]:
# Compute RSS when using squarefeet on TEST data. Handholding level: not as high
train_features = train_data['sqft_living'].values
train_labels = train_data['price'].values
rss_prices_on_sqft = get_residual_sum_of_squares(train_features, train_labels, sqft_intercept, sqft_slope)
print('The RSS of predicting Prices based on Square Feet is : %.2E' % (rss_prices_on_sqft))

The RSS of predicting Prices based on Square Feet is : 1.21E+15


In [ ]:
print('The RSS of predicting Prices based on Square Feet is : %.4f' % (rss_prices_on_sqft))

The RSS of predicting Prices based on Square Feet is : 1209822763116213.5000


# Self Implementation


## Import Dataset


In [ ]:
import numpy as np
import pandas as pd
import sklearn

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/FUNIX Progress/MLP302x_1.1-A_EN/data/kc_house_data.csv')

In [ ]:
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, train_size = 0.8, test_size = 0.2, random_state = 0)

## Close Form Approach


### Feature Extraction

In [ ]:
from scipy import stats

def correlation_approach(input, output):
  pearson_r = stats.pearsonr(input, output)[0]

  std_X = np.std(input)
  std_Y = np.std(output)

  mean_X = input.mean()
  mean_Y = output.mean()

  slope = pearson_r * (std_Y / std_X)
  intercept = mean_Y - slope * mean_X

  return intercept, slope

In [ ]:
def sum_approach(input, output):
  N = len(input)

  sum_XY = np.dot(input, output)
  sum_X = sum(input)
  sum_Y = sum(output)

  mean_X = input.mean()
  mean_Y = output.mean()

  sum_square_X = np.dot(input, input)

  slope = (sum_XY - 1/N * sum_X * sum_Y) / (sum_square_X - 1/N * sum_X * sum_X)
  intercept = mean_Y - slope * mean_X

  return intercept, slope

In [ ]:
def mean_approach(input, output):
  N = len(input)

  mean_XY = (input * output).mean()

  mean_X = input.mean()
  mean_Y = output.mean()

  mean_square_X = (input * input).mean()

  slope = (mean_XY - mean_X * mean_Y) / (mean_square_X - mean_X * mean_X)
  intercept = mean_Y - slope * mean_X

  return intercept, slope

### Model Apply

In [ ]:
def predict(intercept, slope, input):
  return intercept + slope * input

In [ ]:
def rss(intercept, slope, input, output):
  return sum((output - predict(intercept, slope, input)) ** 2)

### Testing Section

In [ ]:
mock_feature = np.array(range(5))
mock_output = 1 + 1*mock_feature
(mock_intercept, mock_slope) = sum_approach(mock_feature, mock_output)
print("Intercept: %.2f" % (mock_intercept))
print("Slope: %.2f" % (mock_slope))

Intercept: 1.00
Slope: 1.00


In [ ]:
mock_feature = np.array(range(5))
mock_output = 1 + 1*mock_feature
(mock_intercept, mock_slope) = mean_approach(mock_feature, mock_output)
print("Intercept: %.2f" % (mock_intercept))
print("Slope: %.2f" % (mock_slope))

Intercept: 1.00
Slope: 1.00


In [ ]:
input = df['sqft_living'].values
output = df['price'].values

intercept, slope = mean_approach(input, output)
print(intercept)
print(slope)

print(predict(intercept, slope, input)[0])
print(output[0])

print(rss(intercept, slope, input, output))

-43580.74309447373
280.62356789744814
287555.06702451507
221900.0
1477276362322495.0


In [ ]:
intercept, slope = correlation_approach(input, output)
print(intercept)
print(slope)

-43580.74309447408
280.6235678974483


## Gradient Descent

### Function Implementation

In [ ]:
def modulus(vector):
  return np.sqrt(vector.dot(vector))

In [ ]:
def error(prediction, label):
  return prediction - label

In [ ]:
def intercept_gradient(error):
  return sum(error)

In [78]:
def slope_gradient(error, input):
  return error.dot(input)

In [ ]:
def rss_gradient(intercept_gradient, slope_gradient):
  return np.array([intercept_gradient, slope_gradient])

In [ ]:
def gradient_descent(input, output, initial_intercept, initial_slope, step_size, tolerance):
  intercept = np.array(initial_intercept)
  slope = np.array(initial_slope)

  magnitude = tolerance + 1

  while(magnitude >= tolerance):
    prediction = predict(intercept, slope, input)
    err = error(prediction, output)

    grad_intercept = intercept_gradient(err)
    grad_slope = slope_gradient(err, input)

    intercept = intercept - step_size * grad_intercept
    slope = slope - step_size * grad_slope

    magnitude = modulus(rss_gradient(grad_intercept, grad_slope))

    # print("- Prediction:", prediction)
    # print("- Error:", err)
    # print("- Intercept:", intercept)
    # print("- Slope:", slope)
    # print("- Magnitude:", magnitude)
    # print("")

  return intercept, slope

#### Testing Section

In [ ]:
initial_intercept = 0
initial_slope = 0
step_size = 0.05
tolerance = 0.01

input = np.array(range(5))
output = np.array([1, 3, 7, 13, 21])

intercept, slope = gradient_descent(input, output, initial_intercept, initial_slope, step_size, tolerance)

print(intercept, slope)

-0.9942069818917416 4.997967918970868


In [79]:
initial_intercept = -47000
initial_slope = 1
step_size = 7e-12
tolerance = 2.5e7

input = train['sqft_living'].values
output = train['price'].values

intercept, slope = gradient_descent(input, output, initial_intercept, initial_slope, step_size, tolerance)
print(intercept, slope)

-46999.88700248911 283.4638130669731
